In [11]:
import pandas as pd
import datasets
import torchvision.transforms as transforms
import torch

In [12]:
data = pd.read_csv('./toy_dataset_label.csv',sep='\t',encoding='latin-1')
print(f'Columns: {[x for x in data.columns]}')

Columns: ['ID', 'FILE', 'AUTHOR', 'BORN-DIED', 'TITLE', 'DATE', 'TECHNIQUE', 'LOCATION', 'FORM', 'TYPE', 'SCHOOL', 'TIMELINE', 'URL']


In [13]:
data.head()

,ID,FILE,AUTHOR,BORN-DIED,TITLE,DATE,TECHNIQUE,LOCATION,FORM,TYPE,SCHOOL,TIMELINE,URL
0,1,1.jpg,"AACHEN, Hans von","(b. 1552, KÃ¶ln, d. 1615, Praha)",Allegory,1598,"Oil on copper, 56 x 47 cm","Alte Pinakothek, Munich",painting,mythological,German,1601-1650,http://www.wga.hu/html/a/aachen/allegory.html
1,2,2.jpg,"AACHEN, Hans von","(b. 1552, KÃ¶ln, d. 1615, Praha)","Bacchus, Ceres and Cupid",-,"Oil on canvas, 163 x 113 cm","Kunsthistorisches Museum, Vienna",painting,mythological,German,1601-1650,http://www.wga.hu/html/a/aachen/bacchus.html
2,3,3.jpg,"AACHEN, Hans von","(b. 1552, KÃ¶ln, d. 1615, Praha)",Joking Couple,-,"Copperplate, 25 x 20 cm","Kunsthistorisches Museum, Vienna",painting,genre,German,1601-1650,http://www.wga.hu/html/a/aachen/j_couple.html
3,4,4.jpg,"AACHEN, Hans von","(b. 1552, KÃ¶ln, d. 1615, Praha)",Portrait of Emperor Rudolf II,1590s,"Oil on canvas, 60 x 48 cm","Kunsthistorisches Museum, Vienna",painting,portrait,German,1601-1650,http://www.wga.hu/html/a/aachen/rudolf2.html
4,5,5.jpg,"AACHEN, Hans von","(b. 1552, KÃ¶ln, d. 1615, Praha)",Self-Portrait with a Glass of Wine,c. 1596,"Oil on canvas, 53 x 44 cm",Private collection,painting,portrait,German,1601-1650,http://www.wga.hu/html/a/aachen/selfport.html


In [14]:
data['TECHNIQUE'] = data['TECHNIQUE'].str.split(',').str[0]
data['TECHNIQUE'] = data['TECHNIQUE'].str.split(':').str[0]

In [15]:
data = data[:700]

In [16]:
from tqdm import tqdm

with tqdm(total=len(data)) as pbar:
    for idx in range(data.shape[0]):
        pbar.set_description(f'Art Piece N°: {idx}')
        data['FILE'][idx] = './images/'+str(idx+1)+'.jpg'
        pbar.update()

Art Piece N°: 1:   0%|          | 1/700 [00:00<00:01, 497.01it/s]C:\Users\rullo\AppData\Local\Temp\ipykernel_11896\3233884474.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['FILE'][idx] = './images/'+str(idx+1)+'.jpg'
Art Piece N°: 699: 100%|██████████| 700/700 [00:01<00:00, 555.12it/s]


In [17]:
data.head()

,ID,FILE,AUTHOR,BORN-DIED,TITLE,DATE,TECHNIQUE,LOCATION,FORM,TYPE,SCHOOL,TIMELINE,URL
0,1,./images/1.jpg,"AACHEN, Hans von","(b. 1552, KÃ¶ln, d. 1615, Praha)",Allegory,1598,Oil on copper,"Alte Pinakothek, Munich",painting,mythological,German,1601-1650,http://www.wga.hu/html/a/aachen/allegory.html
1,2,./images/2.jpg,"AACHEN, Hans von","(b. 1552, KÃ¶ln, d. 1615, Praha)","Bacchus, Ceres and Cupid",-,Oil on canvas,"Kunsthistorisches Museum, Vienna",painting,mythological,German,1601-1650,http://www.wga.hu/html/a/aachen/bacchus.html
2,3,./images/3.jpg,"AACHEN, Hans von","(b. 1552, KÃ¶ln, d. 1615, Praha)",Joking Couple,-,Copperplate,"Kunsthistorisches Museum, Vienna",painting,genre,German,1601-1650,http://www.wga.hu/html/a/aachen/j_couple.html
3,4,./images/4.jpg,"AACHEN, Hans von","(b. 1552, KÃ¶ln, d. 1615, Praha)",Portrait of Emperor Rudolf II,1590s,Oil on canvas,"Kunsthistorisches Museum, Vienna",painting,portrait,German,1601-1650,http://www.wga.hu/html/a/aachen/rudolf2.html
4,5,./images/5.jpg,"AACHEN, Hans von","(b. 1552, KÃ¶ln, d. 1615, Praha)",Self-Portrait with a Glass of Wine,c. 1596,Oil on canvas,Private collection,painting,portrait,German,1601-1650,http://www.wga.hu/html/a/aachen/selfport.html


In [18]:
features = ['FILE','AUTHOR']

train_data = data[features]
train_data.dropna()
train_data = train_data[train_data['AUTHOR'] != 0]
train_data.head()


,FILE,AUTHOR
0,./images/1.jpg,"AACHEN, Hans von"
1,./images/2.jpg,"AACHEN, Hans von"
2,./images/3.jpg,"AACHEN, Hans von"
3,./images/4.jpg,"AACHEN, Hans von"
4,./images/5.jpg,"AACHEN, Hans von"


In [19]:
train_data['AUTHOR'].value_counts()

ANDREA DEL SARTO                62
ANDREA DEL CASTAGNO             51
ANDREA DA FIRENZE               46
ALTDORFER, Albrecht             37
AMMANATI, Bartolomeo            28
                                ..
ADOLPHI, Christoffel             1
ALTOBELLO, Francesco Antonio     1
ADLER, Salomon                   1
AMEDEI, Giuliano                 1
ANGELICO, Fra                    1
Name: AUTHOR, Length: 99, dtype: int64

In [24]:
labels = train_data['AUTHOR'].unique()
train_data = train_data.rename(columns={'FILE':'image', 'AUTHOR':'label'})


In [44]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
size = (250,250)
_transforms = Compose([RandomResizedCrop(size),ToTensor(), normalize])

In [45]:
def transforms(examples):
    examples['pixel_values'] = [_transforms(img.convert('RGB'))for img in examples['image']]
    del examples['image']
    return examples

In [46]:
from torchvision.transforms import Compose
batch_size=32
train_dataset = datasets.Dataset.from_dict(train_data).cast_column('image',datasets.Image())
train_dataset = train_dataset.with_transform(transforms)

In [47]:
train_dataloader=torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [75]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(55696,32)
        self.fc2 = nn.Linear(32,32)
        self.fc3 = nn.Linear(32,32)
    
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [76]:
import torch.optim as optim
net = CNN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 1e-5, momentum=0.9)
device = 'cuda'


In [77]:
for epoch in range(10):
    running_loss = 0.0
    for idx, data in enumerate(train_dataloader):
        label = data.pop('label')
        inputs = data.pop('pixel_values')
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if idx % 2000 == 1999:    # print every 2000 mini-batches
                print(f'[{epoch + 1}, {idx + 1:5d}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0

print('Done!')

TypeError: cross_entropy_loss(): argument 'target' (position 2) must be Tensor, not str